# Property Calculations (MDTraj)

In [1]:
# Cusotm imports
from polymer_utils import analysis, filetree, general
from polymer_utils.representation import PolymerDir, PolymerDirManager
from polymer_utils.graphics import plotutils

# General imports
from pathlib import Path
from datetime import datetime

# Numeric processing and plotting
import mdtraj as mdt
import numpy as np

# Units
from openmm.unit import nanometer, dimensionless

## Load trajectory and calculate/plot properties

In [ ]:
POLY_PATH = Path('Polymers')
SAMPLE_INTERVAL = 10

mgr = PolymerDirManager(POLY_PATH)
pdir = mgr.mol_dirs['polyethylmethacrylate_solv_water']
dated_sims = sorted(pdir.completed_sims, key = lambda sim_dir : datetime.strptime(sim_dir.stem, general.DATETIME_FMT))
# sim_dir_to_use = dated_sims[0] # oldest 
sim_dir_to_use = dated_sims[-1] # most recent

dated_sims

In [ ]:
topo_path = pdir.structure_file
traj_path = next(sim_dir_to_use.glob('*.pdb'))

traj = mdt.load(traj_path, top=topo_path, stride=SAMPLE_INTERVAL)
traj = traj.remove_solvent(inplace=True) # don't generate new copy when de-solvating
topo = traj.topology

In [ ]:
props = [
    analysis.PolyProp(calc=mdt.compute_rg                , name='Radius of Gyration'             , abbr='Rg'  , unit=nanometer),
    analysis.PolyProp(calc=mdt.shrake_rupley             , name='Solvent Accessible Surface Area', abbr='SASA', unit=nanometer**2, kwargs={'mode' : 'residue'}),
    analysis.PolyProp(calc=mdt.relative_shape_antisotropy, name='Relative Shape Anisotropy'      , abbr='K2'  , unit=dimensionless)    
]

rdfs = analysis.plot_rdfs(       traj                                        , header=f'Pairwise Radial Distribution Functions - {pdir.mol_name}', save_path=None)#sim_dir_to_use/f'RDFs.png')
pdata = analysis.plot_poly_props(traj, props, sample_interval=SAMPLE_INTERVAL, header=f'Polymer Shape and Gyration Properties - {pdir.mol_name}' , save_path=None)#sim_dir_to_use/f'shape_properties.png')
# filetree.startfile(sim_dir_to_use)

In [2]:
POLY_PATH = Path('Polymers')
SAMPLE_INTERVAL = 10

props = [
    analysis.PolyProp(calc=mdt.compute_rg                , name='Radius of Gyration'             , abbr='Rg'  , unit=nanometer),
    analysis.PolyProp(calc=mdt.shrake_rupley             , name='Solvent Accessible Surface Area', abbr='SASA', unit=nanometer**2, kwargs={'mode' : 'residue'}),
    analysis.PolyProp(calc=mdt.relative_shape_antisotropy, name='Relative Shape Anisotropy'      , abbr='K2'  , unit=dimensionless)    
]

to_skip = ['polymethylketone_solv_water', 'polyvinylchloride_solv_water', 'atactic_styrene_solv_water', 'syntactic_styrene_solv_water', 'naturalrubber_solv_water']
data = {}
mgr = PolymerDirManager(POLY_PATH)
for mol_name, sim_dirs in mgr.all_completed_sims.items():
    if mol_name in to_skip:
        continue

    print(mol_name)
    pdir = mgr.mol_dirs[mol_name]
    topo_path = pdir.structure_file
    for sim_dir in sim_dirs:
        print(sim_dir.name)
        traj_path = next(sim_dir.glob('*.pdb'))

        traj = mdt.load(traj_path, top=topo_path, stride=SAMPLE_INTERVAL)
        traj = traj.remove_solvent(inplace=True) # don't generate new copy when de-solvating
        topo = traj.topology

        rdfs = analysis.plot_rdfs(       traj                                        , header=f'Pairwise Radial Distribution Functions - {pdir.mol_name}', save_path=sim_dir/f'RDFs.png')
        pdata = analysis.plot_poly_props(traj, props, sample_interval=SAMPLE_INTERVAL, header=f'Polymer Shape and Gyration Properties - {pdir.mol_name}' , save_path=sim_dir/f'shape_properties.png')
        # filetree.startfile(sim_dir_to_use)
        data[mol_name] = (rdfs, pdata)

polythiophene_solv_water
03-31-2023_at_22-12-02_PM


## Some other stuff